### Bitcoin block sampler
[Bitcoin core RPC API](https://en.bitcoin.it/wiki/Original_Bitcoin_client/API_calls_list)

#### Operation ordering


In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is running!')

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}')
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')
# commands = [ ['getblockhash', height] for height in range(619300, 619317) ]
# block_hashes = rpc_connection.batch_(commands)
# import pprint
# pprint.pprint(block_hashes)

In [ ]:
%%time
rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}')
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)

sheight = float('inf')
eheight = 0

start = datetime.datetime(2018, 1, 1, 0, 0, 0, tzinfo=tz_utc)
end = datetime.datetime(2018, 12, 31, 23, 59, 59, tzinfo=tz_utc)

bi = 0
term = 1000
for i in range(term, best_block["height"]+term, term):
    i = min(best_block['height']+1, i)
    commands = [ ['getblockhash', height] for height in range(bi, i) ]
    block_hashes = rpc_connection.batch_(commands)
    commands = [ ['getblock', h] for h in block_hashes ]
    blocks = rpc_connection.batch_(commands)
    ntx = [ block['nTx'] for block in blocks ]

    for block in blocks:
        if get_time(block['time']) < start:
            continue
        elif get_time(block['time']) > end:
            continue
        if sheight > block['height']:
            sheight = block['height']
        if eheight < block['height']:
            eheight = block['height']
    bi = i
    print(f'Reading {i} / {best_block["height"]} ({i/best_block["height"]*100}%)', end='\r')
print('')
print(f'Query-reply done.')
print(f'Start height: {sheight}, End height: {eheight}')

#### 2018. 01. 01. 00. 00. 00. ~ 2018. 12. 31. 23. 59. 59. UST
- Reading 619329 / 619328 (100.00016146533017%)
- Query-reply done.
- Start height: 501961, End height: 556458
- CPU times: user 2min 18s, sys: 45.3 s, total: 3min 3s
- Wall time: 51min 26s

#### 2018. 01. 01. 0. 0. 0 ~ 2018. 12. 31. 23. 59. 59 KTC
- Start height: 501904
- End height: 556402
